Done By: **Ryan Yeo**

Class: **DAAA/FT/1B/01**

Admin Num: **P2214452**

In [1]:
import numpy as np
import os

# Reformating datasets

In [2]:
# Create directory if it exists print err
try:
    os.mkdir('datasets_cleaned')
except OSError as error:
    print(error)  

[WinError 183] Cannot create a file when that file already exists: 'datasets_cleaned'


### Employment

In [3]:
# Clean employment dataset

# Note:
# We cannot use genfromtxt directly because it reads commas contained in strings(unlike csvreader and pd.read_csv) 
# To avoid that, we first read in all the data seperated by a newline before processing it

dirtyData_17to19 = np.genfromtxt('datasets_src/employment/emp_17to19.csv', dtype="U64",delimiter="\n")
dirtyData_19to21 = np.genfromtxt('datasets_src/employment/emp_19to21.csv', dtype="U64",delimiter="\n")

In [4]:
# Convert and clean the dataset so that it can be written to datasets_cleaned
def cleanData(dirtyArr):
    employ_arr = []
    for i in dirtyArr:
        _ = []
        inQuotes = False
        for j,n in enumerate(i):
            if n=='\"':
                # If opening quotes => True elif closing quotes => False
                inQuotes=not inQuotes
            if n==',' and inQuotes:
                # If it's used in a string, change it to a backtick
                # This is for the sole purpose of not causing any error when reading as csv
                # When printing from this column, backticks will be changed back to commas
                if i[j+1]==' ':
                    _.append('`')
                # If it's used in money, (e.g. $3,600) just remove the comma
                else:
                    _.append('')
            else:
                _.append(n)
        employ_arr.append("".join(_))
    employ_arr = np.array(employ_arr)
    return employ_arr


In [5]:
# We can delete one set of 2019 data since we don't need duplicates from both arr
cleaned_17to19 = cleanData(dirtyData_17to19)[:-((len(cleanData(dirtyData_17to19))-1)//3)]

# We can also delete the header for the second arr
cleaned_19to21 = cleanData(dirtyData_19to21)[1:]

cleaned = np.concatenate((cleaned_17to19,cleaned_19to21))

In [6]:
# Now we can write back the data into datasets_cleaned
try:
    os.mkdir('datasets_cleaned/employment')
except OSError as error:
    print(error)  

cleaned.tofile('datasets_cleaned/employment/employ.csv',sep='\n')

[WinError 183] Cannot create a file when that file already exists: 'datasets_cleaned/employment'


However at this point our data is still not in the right format yet

Due to our need to manipulate and remove commas that were not seperators, 
we had to cast each row as a string datatype

When writing to a csv file, it will cause quotation marks to appear for each row
Since we don't want that to affect main.ipynb, we would have to reopen the file the format it

This time since we changed the commas, it would be less of a hassle


In [7]:
f=np.genfromtxt('datasets_cleaned/employment/employ.csv',dtype='U64',delimiter=',')
# Reformat array
f = np.char.replace(f,'\'','')  
f = np.char.replace(f,'\"','')

# We can also remove the '$' and the '%' in the meantime so that we can easily convert into float later
f = np.char.replace(f,'$','')
f = np.char.replace(f,'%','')


# Delete file so that savetxt does not replace chars
os.remove('datasets_cleaned/employment/employ.csv')
# Write it back to csv in the right format now
np.savetxt('datasets_cleaned/employment/employ.csv',f,delimiter=",",fmt='%s')

### Intake
Now we repeat the entire process again for the different intakes (_This includes Poly and Uni_)

In [8]:
# Prior to this, U64 was used and it accidentally cut off data from Poly Intake
dirtyPoly = np.genfromtxt('datasets_src/poly_intake/polytechnics-intake-enrolment-and-graduates-by-course.csv', dtype="U128",delimiter="\n")
dirtyUni = np.genfromtxt('datasets_src/uni_intake/universities-intake-enrolment-and-graduates-by-course.csv',dtype="U128",delimiter="\n")

In [9]:
# Since there is no salary data, we are just replacing commas with backticks using cleanData
cleanPoly = cleanData(dirtyPoly)
cleanUni = cleanData(dirtyUni)

In [10]:
# Poly
try:
    os.mkdir('datasets_cleaned/poly')
except OSError as error:
    print(error)  

# Generates an error if you already have the file opened elsewhere
cleanPoly.tofile('datasets_cleaned/poly/poly_intake.csv',sep='\n')

# Since now each string is one cell of data and not one row, we can go back to using U64
f=np.genfromtxt('datasets_cleaned/poly/poly_intake.csv',dtype='U64',delimiter=',')
# Reformat array
f = np.char.replace(f,'\'','')  
f = np.char.replace(f,'\"','')

# Delete file so that savetxt does not replace chars
os.remove('datasets_cleaned/poly/poly_intake.csv')
# Write it back to csv in the right format now
np.savetxt('datasets_cleaned/poly/poly_intake.csv',f,delimiter=",",fmt='%s')

[WinError 183] Cannot create a file when that file already exists: 'datasets_cleaned/poly'


In [11]:
# Uni
try:
    os.mkdir('datasets_cleaned/uni')
except OSError as error:
    print(error)  

# Generates an error if you already have the file opened elsewhere
cleanUni.tofile('datasets_cleaned/uni/uni_intake.csv',sep='\n')

f=np.genfromtxt('datasets_cleaned/uni/uni_intake.csv',dtype='U64',delimiter=',')
# Reformat array
f = np.char.replace(f,'\'','')  
f = np.char.replace(f,'\"','')

# Delete file so that savetxt does not replace chars
os.remove('datasets_cleaned/uni/uni_intake.csv')
# Write it back to csv in the right format now
np.savetxt('datasets_cleaned/uni/uni_intake.csv',f,delimiter=",",fmt='%s')

[WinError 183] Cannot create a file when that file already exists: 'datasets_cleaned/uni'


# Cleaning Datasets

### Employment

In [12]:
employEdit = np.genfromtxt('datasets_cleaned/employment/employ.csv',dtype='U64',delimiter=',')

|       Initial Col Name   | New Col Name |
|----------------------|------|
| Engineering | Engineering |
| Architecture | Architecture and Built Environment|
| Business | Business |
| Information & Digital Technologies | Information & Digital Technologies|
| Medicine | Medicine |
| Arts, Design & Medias | Arts, Humanities and the Social Sciences|
| Dentistry | Dentistry |
| Built environment | Architecture and Built Environment|
|Yale-NUS| *Removed*|
|Biomedical Sciences| Health Sciences |
| Pharmacy | Health Sciences |
| Education (NIE) | Education |
| Music | Arts, Humanities and the Social Sciences|
| Humanities & Social Sciences | Arts, Humanities and the Social Sciences|
| Health Sciences | Health Sciences |
| Sciences | Sciences |
| Law | Law |

In [13]:
# Rename course name to match the other datasets
for a in employEdit:
    if a[1]=='Architecture' or a[1]=='Built Environment':
        a[1] = 'Architecture and Built Environment'
    elif a[1]=='Arts` Design & Media' or a[1]=='Music' or a[1]=='Humanities & Social Sciences':
        a[1] = 'Arts` Humanities and the Social Sciences'
    elif a[1]=='Biomedical Sciences' or a[1]=='Pharmacy' or a[1]=='Health Sciences':
        a[1] = 'Health Sciences'
    elif a[1] == 'Education(NIE)':
        a[1] = 'Education'

# Remove row if it contains Yale-NUS
counter=0
for i,a in enumerate(employEdit):
    if a[1]=='Yale-NUS':
        employEdit = np.delete(employEdit,i-counter,0)
        counter+=1

print(employEdit)


[['year' 'course_cluster' 'employed' 'ft_employment'
  'gross median salary']
 ['2017' 'Arts` Humanities and the Social Sciences' '91.40' '65.60'
  '2944']
 ['2017' 'Architecture and Built Environment' '92.70' '86.50' '3200']
 ['2017' 'Business' '95.40' '89.50' '3200']
 ['2017' 'Dentistry' '100' '100.00' '4050']
 ['2017' 'Education' '100' '100.00' '3600']
 ['2017' 'Engineering' '86.70' '79.50' '3500']
 ['2017' 'Health Sciences' '96.70' '93.70' '3450']
 ['2017' 'Arts` Humanities and the Social Sciences' '85.70' '70.10'
  '3300']
 ['2017' 'Information & Digital Technologies' '94.60' '90.10' '4000']
 ['2017' 'Arts` Humanities and the Social Sciences' '73.30' '26.70'
  '2225']
 ['2017' 'Sciences' '82.60' '65.30' '3250']
 ['2017' 'Architecture and Built Environment' '91.30' '86.40' '4000']
 ['2017' 'Health Sciences' '92.00' '80.00' '2950']
 ['2017' 'Law' '96.40' '92.80' '5000']
 ['2017' 'Medicine' '100.00' '100.00' '5000']
 ['2017' 'Health Sciences' '99.10' '94.50' '3600']
 ['2018' 'Arts` H

In [14]:
# Delete file so that savetxt does not replace chars
os.remove('datasets_cleaned/employment/employ.csv')
# Write it back to csv in the right format now
np.savetxt('datasets_cleaned/employment/employ.csv',employEdit,delimiter=",",fmt='%s')

In [15]:
# Read csv file
employArr = np.genfromtxt('datasets_cleaned/employment/employ.csv',dtype='float',delimiter=',',skip_header=1,usecols=(0,2,3,4))
col_header = np.genfromtxt('datasets_cleaned/employment/employ.csv',dtype='U64',delimiter=',',usecols=(0,2,3,4))[0]
employCourse = np.genfromtxt('datasets_cleaned/employment/employ.csv',dtype='U64',delimiter=',',skip_header=1,usecols=(1))
# Since we have to use isnan, setting names=True is not an option since the string cannot be set to float
# As such we make our own artificial columns using dictionary and 'col_header'
employNames = {}
for i,n in enumerate(col_header):
    employNames[n]=i

# Get all missing data
missing = np.argwhere(np.isnan(employArr))
for i in missing:
# Get col name by swapping key and values inside employNames
    print(f"course cluster: {employCourse[i[0]]}, year: {int(employArr[i[0]][0])}, column: {({v:k for k,v in employNames.items()})[i[1]]}")

course cluster: Medicine, year: 2018, column: employed
course cluster: Medicine, year: 2018, column: ft_employment
course cluster: Medicine, year: 2018, column: gross median salary
course cluster: Medicine, year: 2021, column: employed
course cluster: Medicine, year: 2021, column: ft_employment
course cluster: Medicine, year: 2021, column: gross median salary


As seen above, the missing data is for the course cluster "Medicine" and for year 2018 and 2021

Since:
1. The missing data is not randomly distributed (its just for medicine)
2. They are accounted for by other data in our datasets (Medicine data of salary and employment percentages from year 2017, 2019 and 2021 are likely to be similar to 2017,2019 and 2020)

The missing data is Missing At Random

*In the PDF by MOE, it is explained that the missing data is due to insufficient graduates/response rate*

Since the data is MAR, we can either choose to impute or remove the data

In [16]:
# Mean imputation

# Get data for medicine from 2017,2019 and 2021
dataForImpute = np.array([])
for i,j in zip(employArr,employCourse):
    if i[employNames["year"]] in [2017,2019,2020] and j=='Medicine':
        dataForImpute = np.concatenate((dataForImpute,i[1:]))

print(dataForImpute)

ImputeData = np.zeros((3,3))

# Reformat data(by grouping similar cols together) so that np.mean() can be used
for iter in range(3):
    ImputeData[iter] = np.array([n for i,n in enumerate(dataForImpute) if i%3==iter])

# Replace nan values with mean
for i,n in enumerate(ImputeData):
    employArr[missing[0][0]][i+1] = round(n.mean(),2)
    employArr[missing[3][0]][i+1] = round(n.mean(),2)

print(employArr[missing[0][0]])
print(employArr[missing[3][0]])

# If imputation was done correctly, the value of this should be 0
print(len(np.argwhere(np.isnan(employArr))))

[ 100.   100.  5000.    99.6   99.6 5300.   100.   100.  5250. ]
[2018.     99.87   99.87 5183.33]
[2021.     99.87   99.87 5183.33]
0


### Intake

|       Initial Col Name   | New Col Name |
|----------------------|------|
| Engineering Sciences | Engineering |
| Architecture, Building & Real Estate | Architecture and Built Environment|
| Business & Administration | Business |
| Information Technology | Information & Digital Technologies|
| Applied Arts | Arts, Humanities and the Social Sciences|
| Mass Communication | Arts, Humanities and the Social Sciences|
| Services | Arts, Humanities and the Social Sciences|
| Humanities & Social Sciences | Arts, Humanities and the Social Sciences|
| Health Sciences | Health Sciences |
| Education | Education |
| Natural & Mathematical Sciences | Sciences |
Natural, Physical & Mathematical Sciences | Sciences |
| Law | Law |

In [17]:
polyEdit = np.genfromtxt('datasets_cleaned/poly/poly_intake.csv',dtype='U64',delimiter=',',skip_header=1)
for a in polyEdit:
    if 'Engineering' in a[2]:
        a[2] = 'Engineering'
    elif 'Architecture' in a[2]:
        a[2] = 'Architecture and Built Environment'
    elif 'Business' in a[2]:
        a[2] = 'Business'
    elif 'Information Technology' in a[2]:
        a[2] = 'Information & Digital Technologies'
    elif a[2] == 'Applied Arts' or a[2] == 'Mass Communication' or a[2]=='Services' or 'Humanities' in a[2]:
        a[2] = 'Arts` Humanities and the Social Sciences'
    elif 'Sciences' in a[2] and a[2]!= 'Health Sciences':
        a[2] = 'Sciences'

In [18]:
# remove all gender with MF and replace it with M (by taking values from MF-values from F)
# from 2018 (inclusive and before), the data is stored as follows:
# MF,course1
# F,course1
# MF,course2
# F,course2

# after 2018, the data is stored as follows:
# MF,course1
# MF,course2
# F,course1
# F,course2

for i,a in enumerate(polyEdit):
    if int(a[0])<=2018:
        if a[1]=='MF':
            a[1]='M'
            a[3] = str(int(a[3])-int(polyEdit[i+1][3]))
            a[4] = str(int(a[4])-int(polyEdit[i+1][4]))
            a[5] = str(int(a[5])-int(polyEdit[i+1][5]))
    else:
        if a[1]=='MF':
            a[1]='M'
            # Since there are 12 different courses
            a[3] = str(int(a[3])-int(polyEdit[i+12][3]))
            a[4] = str(int(a[4])-int(polyEdit[i+12][4]))
            a[5] = str(int(a[5])-int(polyEdit[i+12][5]))

# Reformat data for 2019 and 2020
counter = 0
for i in range(len(polyEdit)):
    if int(polyEdit[i][0])==2019:
        if counter<12:
            # We have to use vstack instead of concat or append since we are adding a 1d array to a 2d array (and since axis=0)
            polyEdit = np.vstack((polyEdit[:i+1+counter],polyEdit[i+12+counter],polyEdit[i+1+counter:]))
        # remove duplicates from 2019
        elif counter==24:
            polyEdit = np.delete(polyEdit,np.s_[i:i+12],0)
        counter+=1

counter = 0
for i in range(len(polyEdit)):
    if int(polyEdit[i][0])==2020:
        if counter<12:
            polyEdit = np.vstack((polyEdit[:i+1+counter],polyEdit[i+12+counter],polyEdit[i+1+counter:]))
        else:
            print(polyEdit[i],counter)
        counter+=1

# since the last 12 rows are duplicates, we remove them
polyEdit = np.delete(polyEdit,np.s_[-12:],0)

            

['2020' 'M' 'Arts` Humanities and the Social Sciences' '70' '211' '89'] 12
['2020' 'F' 'Arts` Humanities and the Social Sciences' '228' '698' '259'] 13
['2020' 'M' 'Information & Digital Technologies' '1883' '5824' '1808'] 14
['2020' 'F' 'Information & Digital Technologies' '637' '2173' '802'] 15
['2020' 'M' 'Law' '48' '118' '31'] 16
['2020' 'F' 'Law' '60' '210' '62'] 17
['2020' 'M' 'Arts` Humanities and the Social Sciences' '134' '458' '159'] 18
['2020' 'F' 'Arts` Humanities and the Social Sciences' '415' '1334' '461'] 19
['2020' 'M' 'Sciences' '440' '1090' '471'] 20
['2020' 'F' 'Sciences' '669' '1947' '730'] 21
['2020' 'M' 'Arts` Humanities and the Social Sciences' '416' '1706' '526'] 22
['2020' 'F' 'Arts` Humanities and the Social Sciences' '375' '1222' '482'] 23


In [19]:
polyEdit = np.vstack((np.genfromtxt('datasets_cleaned/poly/poly_intake.csv',dtype='U64',delimiter=',')[0],polyEdit))
print(polyEdit)

[['year' 'sex' 'course' 'intake' 'enrolment' 'graduates']
 ['2005' 'M' 'Arts` Humanities and the Social Sciences' '441' '1055'
  '248']
 ['2005' 'F' 'Arts` Humanities and the Social Sciences' '687' '1538'
  '302']
 ...
 ['2020' 'F' 'Sciences' '669' '1947' '730']
 ['2020' 'M' 'Arts` Humanities and the Social Sciences' '416' '1706'
  '526']
 ['2020' 'F' 'Arts` Humanities and the Social Sciences' '375' '1222'
  '482']]


In [20]:
# Delete file so that savetxt does not replace chars
os.remove('datasets_cleaned/poly/poly_intake.csv')
# Write it back to csv in the right format now
np.savetxt('datasets_cleaned/poly/poly_intake.csv',polyEdit,delimiter=",",fmt='%s')

|       Initial Col Name   | New Col Name |
|----------------------|------|
| Engineering Sciences | Engineering |
| Architecture, Building & Real Estate | Architecture and Built Environment|
| Business & Administration | Business |
| Accountancy | Business |
| Information Technology | Information & Digital Technologies|
| Fine & Applied Arts | Arts, Humanities and the Social Sciences|
| Mass Communication | Arts, Humanities and the Social Sciences|
| Services | Arts, Humanities and the Social Sciences|
| Humanities & Social Sciences | Arts, Humanities and the Social Sciences|
| Medicine | Medicine |
| Dentistry | Dentistry |
| Health Sciences | Health Sciences |
| Education | Education |
| Natural & Mathematical Sciences | Sciences |
Natural, Physical & Mathematical Sciences | Sciences |
| Law | Law |

In [21]:
uniEdit = np.genfromtxt('datasets_cleaned/uni/uni_intake.csv',dtype='U64',delimiter=',',skip_header=1)
for a in uniEdit:
    if 'Engineering' in a[2]:
        a[2] = 'Engineering'
    elif 'Architecture' in a[2]:
        a[2] = 'Architecture and Built Environment'
    elif 'Business' in a[2] or a[2] == 'Accountancy':
        a[2] = 'Business'
    elif 'Information Technology' in a[2]:
        a[2] = 'Information & Digital Technologies'
    elif 'Applied Arts' in a[2] or a[2] == 'Mass Communication' or a[2]=='Services' or 'Humanities' in a[2]:
        a[2] = 'Arts` Humanities and the Social Sciences'
    elif 'Sciences' in a[2] and a[2]!= 'Health Sciences':
        a[2] = 'Sciences'

In [22]:
for i,a in enumerate(uniEdit):
    if int(a[0])<=2018:
        if a[1]=='MF':
            a[1]='M'
            a[3] = str(int(a[3])-int(uniEdit[i+1][3]))
            a[4] = str(int(a[4])-int(uniEdit[i+1][4]))
            a[5] = str(int(a[5])-int(uniEdit[i+1][5]))
    else:
        if a[1]=='MF':
            a[1]='M'
            # Since there are 12 different courses
            a[3] = str(int(a[3])-int(uniEdit[i+15][3]))
            a[4] = str(int(a[4])-int(uniEdit[i+15][4]))
            a[5] = str(int(a[5])-int(uniEdit[i+15][5]))

# Reformat data for 2019 and 2020
counter = 0
for i in range(len(uniEdit)):
    if int(uniEdit[i][0])==2019:
        if counter<15:
            # We have to use vstack instead of concat or append since we are adding a 1d array to a 2d array (and since axis=0)
            uniEdit = np.vstack((uniEdit[:i+1+counter],uniEdit[i+15+counter],uniEdit[i+1+counter:]))
        # remove duplicates from 2019
        elif counter==30:
            uniEdit = np.delete(uniEdit,np.s_[i:i+15],0)
        counter+=1

counter = 0
for i in range(len(uniEdit)):
    if int(uniEdit[i][0])==2020:
        if counter<15:
            uniEdit = np.vstack((uniEdit[:i+1+counter],uniEdit[i+15+counter],uniEdit[i+1+counter:]))
        else:
            print(uniEdit[i],counter)
        counter+=1

# since the last 12 rows are duplicates, we remove them
uniEdit = np.delete(uniEdit,np.s_[-15:],0)

['2020' 'F' 'Health Sciences' '761' '2455' '522'] 15
['2020' 'M' 'Arts` Humanities and the Social Sciences' '929' '4109' '1022'] 16
['2020' 'F' 'Arts` Humanities and the Social Sciences' '2001' '8783'
 '2125'] 17
['2020' 'M' 'Information & Digital Technologies' '2017' '6014' '970'] 18
['2020' 'F' 'Information & Digital Technologies' '846' '2797' '397'] 19
['2020' 'M' 'Law' '204' '916' '190'] 20
['2020' 'F' 'Law' '259' '907' '204'] 21
['2020' 'M' 'Arts` Humanities and the Social Sciences' '28' '148' '39'] 22
['2020' 'F' 'Arts` Humanities and the Social Sciences' '142' '566' '142'] 23
['2020' 'M' 'Medicine' '253' '1158' '208'] 24
['2020' 'F' 'Medicine' '224' '1020' '178'] 25
['2020' 'M' 'Sciences' '788' '3089' '801'] 26
['2020' 'F' 'Sciences' '891' '3624' '1127'] 27
['2020' 'M' 'Arts` Humanities and the Social Sciences' '143' '402' '107'] 28
['2020' 'F' 'Arts` Humanities and the Social Sciences' '139' '412' '112'] 29


In [23]:
uniEdit = np.vstack((np.genfromtxt('datasets_cleaned/uni/uni_intake.csv',dtype='U64',delimiter=',')[0],uniEdit))
print(uniEdit)

[['year' 'sex' 'course' 'intake' 'enrolment' 'graduates']
 ['2005' 'M' 'Business' '346' '829' '211']
 ['2005' 'F' 'Business' '530' '1732' '495']
 ...
 ['2020' 'F' 'Sciences' '891' '3624' '1127']
 ['2020' 'M' 'Arts` Humanities and the Social Sciences' '143' '402' '107']
 ['2020' 'F' 'Arts` Humanities and the Social Sciences' '139' '412' '112']]


In [24]:
# Delete file so that savetxt does not replace chars
os.remove('datasets_cleaned/uni/uni_intake.csv')
# Write it back to csv in the right format now
np.savetxt('datasets_cleaned/uni/uni_intake.csv',uniEdit,delimiter=",",fmt='%s')

In [25]:
# Combine intakes,enrollment and graduation data for similar courses